In [1]:
from AOC_utils import get_day
import numpy as np

day = 24
input_data = get_day(day)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Day 24 input already downloaded
313


['x00: 1', 'x01: 0', 'x02: 1', 'x03: 1', 'x04: 0', 'x05: 0', 'x06: 1']

In [2]:
example = '''
x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj
'''.split('\n')[1:-1]

In [3]:
example = '''
x00: 0
x01: 1
x02: 0
x03: 1
x04: 0
x05: 1
y00: 0
y01: 0
y02: 1
y03: 1
y04: 0
y05: 1

x00 AND y00 -> z05
x01 AND y01 -> z02
x02 AND y02 -> z01
x03 AND y03 -> z03
x04 AND y04 -> z04
x05 AND y05 -> z00
'''.split('\n')[1:-1]

In [4]:
def solve_day(data, example=False):
    part1 = 0
    part2 = 0

    wires = {}
    gates = []
    for line in data:
        if ':' in line:
            wire, value = line.split(': ')
            wires[wire] = int(value)
        elif '->' in line:
            gate0, op, gate1, _, wire = line.split()
            gates.append(op + " " + gate0 + " " + gate1 + " " + wire)

    # print(wires)

    output_wires = {}
    for gate in gates:
        # if 'z' in gate:
        op, gate0, gate1, wire = gate.split()
        output_wires[wire] = [gate0, gate1]

    zoutput_wires = {key: value for key, value in output_wires.items() if key.startswith('z')}
    for key, value in zoutput_wires.items():
        for v in value:
            if v in output_wires:
                for z in output_wires[v]:
                    if z not in zoutput_wires[key]:
                        zoutput_wires[key].append(z)
    zkeys = list(zoutput_wires.keys())
    zvalues = list(zoutput_wires.values())
    zorder = np.argsort(zkeys)
    zkeys = np.array(zkeys)[zorder].tolist()
    zvalues = np.array(zvalues)[zorder].tolist()
    zoutput_wires = dict(zip(zkeys[::-1], zvalues[::-1]))

    # print(zoutput_wires)
    # for key, value in zoutput_wires.items():
    #     print(key, len(value))

    Xn = []
    Yn = []
    for key, value in wires.items():
        if key.startswith('x'):
            Xn.append((key, value))
        if key.startswith('y'):
            Yn.append((key, value))
    Xn = sorted(Xn, key=lambda x: x[0], reverse=True)
    Yn = sorted(Yn, key=lambda x: x[0], reverse=True)
    Xn = [str(x[1]) for x in Xn]
    Yn = [str(x[1]) for x in Yn]
    Xn = int("".join(Xn), 2)
    Yn = int("".join(Yn), 2)

    # print(Xn, Yn)

    def solve(og_wires, og_gates):
        wires = og_wires.copy()
        gates = og_gates.copy()
        while gates:
            gate = gates.pop(0)
            op, gate0, gate1, wire = gate.split(' ')
            if gate0 in wires and gate1 in wires:
                if op == 'AND':
                    wires[wire] = wires[gate0] & wires[gate1]
                elif op == 'OR':
                    wires[wire] = wires[gate0] | wires[gate1]
                elif op == 'XOR':
                    wires[wire] = wires[gate0] ^ wires[gate1]
                else:
                    print("unknown op:", op)
            else:
                gates.append(gate)

        output_wires = []
        for key, value in wires.items():
            if key.startswith('z'):
                output_wires.append((key, value))

        output_wires = sorted(output_wires, key=lambda x: x[0], reverse=True)
        output_nums = [str(x[1]) for x in output_wires]
        return output_nums
    
    intitial_solve = solve(wires, gates)
    bin_solve = "".join(intitial_solve)
    part1 = int(bin_solve, 2)

    # for _ in range(10000):
    #     _ = solve(wires, gates)

    # find problem bits
    problem_bits = 0
    for i in range(len(bin_solve)):
        if example:
            new_Xn = {'x'+"{:02d}".format(x): 1 for x in range(i)}
            new_Xn.update({'x'+"{:02d}".format(x): 0 for x in range(i, len(bin_solve))})
            new_Yn = {'y'+"{:02d}".format(x): 1 for x in range(i)}
            new_Yn.update({'y'+"{:02d}".format(x): 0 for x in range(i, len(bin_solve))})
        else:
            new_Xn = {'x'+"{:02d}".format(x): 0 for x in range(i)}
            new_Xn.update({'x'+"{:02d}".format(x): 1 for x in range(i, len(bin_solve))})
            new_Yn = {'y'+"{:02d}".format(x): 1 for x in range(i)}
            new_Yn.update({'y'+"{:02d}".format(x): 0 for x in range(i, len(bin_solve))})
            
        new_wires = wires.copy()
        for key, value in new_Xn.items():
            if key in new_wires:
                new_wires[key] = value
        for key, value in new_Yn.items():
            if key in new_wires:
                new_wires[key] = value

        Xnum = int(''.join([str(x) for x in new_Xn.values()][::-1]), 2)
        Ynum = int(''.join([str(x) for x in new_Yn.values()][::-1]), 2)

        correct_answer = Xnum & Ynum if example else Xnum + Ynum

        new_output = solve(new_wires, gates)
        new_bin = "".join(new_output)

        new_problem_bits = correct_answer ^ int(new_bin, 2)
        if len(bin(new_problem_bits)[2:]) > len(bin_solve):
            continue
        problem_bits = problem_bits | new_problem_bits

    # if problem bits are not the same length as the output, add leading zeros
    problem_bits = bin(problem_bits)[2:]
    if len(problem_bits) < len(bin_solve):
        problem_bits = '0'*(len(bin_solve) - len(problem_bits)) + problem_bits
    print(problem_bits)

    # for key, value in zoutput_wires.items():
    #     print(key, len(value), value)


    print("part 1:", part1)
    print("part 2:", part2)


solve_day(example, example=True)

100111
part 1: 9
part 2: 0


In [5]:
solve_day(input_data)

0111111000000000000000011111111111111111000000
part 1: 57632654722854
part 2: 0


C:\Users\Alex\AppData\Local\Temp\ipykernel_18696\3230600643.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  zvalues = np.array(zvalues)[zorder].tolist()
